In [1]:
import numpy as np
from qiskit import *
from qiskit.tools.visualization import plot_histogram

In [4]:
location = QuantumRegister(8)
coin = QuantumRegister(1)
c = ClassicalRegister(8)

circuit = QuantumCircuit(location, coin, c)
circuit.x(location[0:3])
circuit.mct(location[0:3], location[7], location[3:6])
# circuit.x(location[7])

# for i in range(40):
#     circuit.h(coin[0])
#     TakeStep(coin, location, circuit)

circuit.measure(range(8), range(8))
circuit.draw()

# localSimulator = Aer.get_backend('qasm_simulator')
# result = execute(circuit, localSimulator, shots=100000).result()
# counts = result.get_counts(circuit)
# plot_histogram(counts)

┌───┐                                                        »
q4_0: |0>───┤ X ├────────────────────────────────────■───────────────────»
            ├───┤                                    │                   »
q4_1: |0>───┤ X ├──────────────────■─────────────────┼────────────────■──»
            ├───┤                  │                 │                │  »
q4_2: |0>───┤ X ├──────────────────┼─────────────────┼────────────────┼──»
         ┌──┴───┴───┐┌──────────┐┌─┴─┐┌───────────┐┌─┴─┐┌──────────┐┌─┴─┐»
q4_3: |0>┤ U2(0,pi) ├┤ U1(pi/4) ├┤ X ├┤ U1(-pi/4) ├┤ X ├┤ U1(pi/4) ├┤ X ├»
         └───┬─┬────┘└──────────┘└───┘└───────────┘└───┘└──────────┘└───┘»
q4_4: |0>────┤M├─────────────────────────────────────────────────────────»
             └╥┘         ┌─┐                                             »
q4_5: |0>─────╫──────────┤M├─────────────────────────────────────────────»
              ║          └╥┘      ┌─┐                                    »
q4_6: |0>─────╫───────────╫───────┤M├────────────────────────────────────»
              ║           ║       └╥┘                                    »
q4_7: |0>─────╫───────────╫────────╫─────────────────────────────────────»
              ║           ║        ║                                     »
q5_0: |0>─────╫───────────╫────────╫─────────────────────────────────────»
              ║           ║        ║                                     »
 c2_0: 0 ═════╬═══════════╬════════╬═════════════════════════════════════»
              ║           ║        ║                                     »
 c2_1: 0 ═════╬═══════════╬════════╬═════════════════════════════════════»
              ║           ║        ║                                     »
 c2_2: 0 ═════╬═══════════╬════════╬═════════════════════════════════════»
              ║           ║        ║                                     »
 c2_3: 0 ═════╬═══════════╬════════╬═════════════════════════════════════»
              ║           ║        ║                                     »
 c2_4: 0 ═════╩═══════════╬════════╬═════════════════════════════════════»
                          ║        ║                                     »
 c2_5: 0 ═════════════════╩════════╬═════════════════════════════════════»
                                   ║                                     »
 c2_6: 0 ══════════════════════════╩═════════════════════════════════════»
                                                                         »
 c2_7: 0 ════════════════════════════════════════════════════════════════»
                                                                         »
«                                                                    »
«q4_0: ──────────────────────────────────────────────────────────────»
«                                                                    »
«q4_1: ───────────────────────────────────────────────────────────■──»
«                                                ┌─┐              │  »
«q4_2: ───────────────────────────■──────────────┤M├──────────────┼──»
«      ┌───────────┐┌──────────┐  │  ┌──────────┐└╥┘┌──────────┐┌─┴─┐»
«q4_3: ┤ U1(-pi/4) ├┤ U2(0,pi) ├──■──┤ U2(0,pi) ├─╫─┤ U1(pi/4) ├┤ X ├»
«      └───────────┘└──────────┘  │  └──────────┘ ║ └──────────┘└───┘»
«q4_4: ───────────────────────────┼───────────────╫──────────────────»
«                                 │               ║                  »
«q4_5: ───────────────────────────┼───────────────╫──────────────────»
«                                 │               ║                  »
«q4_6: ───────────────────────────┼───────────────╫──────────────────»
«                               ┌─┴─┐    ┌─┐      ║                  »
«q4_7: ─────────────────────────┤ X ├────┤M├──────╫──────────────────»
«                               └───┘    └╥┘      ║                  »
«q5_0: ───────────────────────────────────╫───────╫──────────────────»
«                                         ║       ║                  »
«c2_0: ═══════════════════════════════

In [4]:
def TakeStep(coinReg, locationReg, circuit):
    circuit.x(coinReg[0])
    ControlledIncrement(-1, coinReg[0], locationReg, circuit)
    circuit.x(coinReg[0])
    ControlledIncrement(1, coinReg[0], locationReg, circuit)

In [5]:
def Increment(increment, quantumRegister, circuit):
    from qiskit.aqua.components.qfts import Standard as qft
    from qiskit.aqua.components.iqfts import Standard as iqft
    
    n = len(quantumRegister)
    
    qft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
    for j in range(n):
        circuit.u1((np.pi*increment)/(2**(n-1-j)), quantumRegister[j])
        
    iqft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
def ControlledIncrement(increment, controlQubit, quantumRegister, circuit):
    from qiskit.aqua.components.qfts import Standard as qft
    from qiskit.aqua.components.iqfts import Standard as iqft
    
    n = len(quantumRegister)
    
    qft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
    for j in range(n):
        circuit.cu1((np.pi*increment)/(2**(n-1-j)), controlQubit, quantumRegister[j])
        
    iqft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)